In [29]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from implementations_tanguy import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
from helpers import *

# load data.
particle, products, ids = load_csv_data('data/train.csv', sub_sample= True)

In [31]:
# build sampled x and y.
seed = 1
y = np.expand_dims(particle, axis=1)
y, X = sample_data(y, products, seed, size_samples=150)
x, mean_x, std_x = standardize(X)

tx = np.c_[np.ones((y.shape[0], 1)), x]

### LEAST SQUARES WITH NORMAL EQUATIONS

In [32]:
print(y.shape, x.shape)
print(least_squares(y,tx)[1])

(150, 1) (150, 30)
[[0.27040399]]


### Logistic Regression

In [95]:
np.seterr(divide = 'ignore') 

initial_w = np.zeros((tx.shape[1], 1))

print("loss", logistic_regression(y,tx, initial_w, 1000, 0.0001)[1][0][0])


#the loss becomes negative if we choose to big of a gamma and number of iterations
#it seems that at some point we go past the global minimum and start diverging 
#having a very small gamma solves this problem

loss 0.3705442180398307
